In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!nvidia-smi

Sat Jan 25 17:59:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:

import pandas as pd
import numpy as np

expander = pd.read_csv('gdrive/My Drive/Expander_data.csv')


Checking if the Columns have Null values. Looks good, only one column.

In [0]:
expander.columns[expander.isna().any()]

Index(['Run Status'], dtype='object')

Attribute of interest (The targets) are stored in y variable.

In [0]:
y = expander.iloc[:,6]
expander = expander.drop("Discharge Pressure (psig)",axis=1)

In [0]:
expander.head(5)

,Timestamp,Suction Pressure (psig),Suction Temperature (F),Total Flow (gpm),Speed (rpm),By-pass Valve Position (%),Discharge Temperature (F),Run Status
0,1/4/2020 21:20,596.100853,-229.436683,7925.800487,2999.411045,6.0,-227.614448,NaN
1,1/4/2020 21:20,596.838217,-230.453708,8085.917724,2989.706617,6.0,-228.668636,NaN
2,1/4/2020 21:20,596.468895,-229.545356,7902.175243,3006.331679,6.0,-227.755324,NaN
3,1/4/2020 21:20,603.568271,-230.309438,7991.808959,3010.487486,6.0,-228.489113,NaN
4,1/4/2020 21:21,595.407372,-229.767969,7871.560747,2986.665518,6.0,-227.971795,NaN


Some things we can infer:
1. Run Status has no useful value, hence can be dropped.
2. Discharge temperature and Suction Temperature has negative values only. 

In [0]:
expander.describe()

,Suction Pressure (psig),Suction Temperature (F),Total Flow (gpm),Speed (rpm),By-pass Valve Position (%),Discharge Temperature (F),Run Status
count,296450.000000,296450.000000,296450.000000,296450.000000,296450.000000,296450.000000,0.0
mean,590.541350,-231.564934,8119.917777,2955.416695,2.005667,-229.764934,NaN
std,23.216552,3.574107,398.049531,54.201467,2.488568,3.574136,NaN
min,552.420048,-238.594971,7056.004500,2835.750571,-0.200000,-236.816223,NaN
25%,572.757269,-234.171427,7916.322054,2906.336126,0.000000,-232.371760,NaN
50%,586.491792,-230.922352,8144.161813,2961.788792,0.000000,-229.122946,NaN
75%,605.986608,-229.585168,8388.610374,3000.863611,4.000000,-227.785796,NaN
max,649.217417,-223.440125,8975.986378,3090.367995,8.026667,-221.617102,NaN


For better understanding of the data, lets convert the negative values to positive values.

In [0]:
def neg_to_positive(data,col):
  min_val = min(data[col].values)
  min_val = abs(min_val)+1
  expander[col] += min_val 

neg_to_positive(expander,"Suction Temperature (F)")
neg_to_positive(expander,"Discharge Temperature (F)")

In [0]:
print(min(expander["Suction Temperature (F)"].values), max(expander["Suction Temperature (F)"].values))
print(min(expander["Discharge Temperature (F)"].values), max(expander["Discharge Temperature (F)"].values))

1.0 16.15484619999998
1.0 16.199120399999998


In [0]:
expander.describe()

,Suction Pressure (psig),Suction Temperature (F),Total Flow (gpm),Speed (rpm),By-pass Valve Position (%),Discharge Temperature (F),Run Status
count,296450.000000,296450.000000,296450.000000,296450.000000,296450.000000,296450.000000,0.0
mean,590.541350,8.030037,8119.917777,2955.416695,2.005667,8.051288,NaN
std,23.216552,3.574107,398.049531,54.201467,2.488568,3.574136,NaN
min,552.420048,1.000000,7056.004500,2835.750571,-0.200000,1.000000,NaN
25%,572.757269,5.423544,7916.322054,2906.336126,0.000000,5.444463,NaN
50%,586.491792,8.672619,8144.161813,2961.788792,0.000000,8.693277,NaN
75%,605.986608,10.009802,8388.610374,3000.863611,4.000000,10.030426,NaN
max,649.217417,16.154846,8975.986378,3090.367995,8.026667,16.199120,NaN


Dropping Run Status because it has no useful information.

In [0]:
expander = expander.drop(['Run Status'], axis=1)

Scaling data. 

In [0]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
expander.iloc[:,1:] = scaler.fit_transform(expander.iloc[:,1:])

In [0]:
expander.head(3)

,Timestamp,Suction Pressure (psig),Suction Temperature (F),Total Flow (gpm),Speed (rpm),By-pass Valve Position (%),Discharge Temperature (F)
0,1/4/2020 21:20,0.451260,0.604314,0.453023,0.642770,0.753647,0.605415
1,1/4/2020 21:20,0.458878,0.537205,0.536418,0.604656,0.753647,0.536056
2,1/4/2020 21:20,0.455062,0.597143,0.440718,0.669951,0.753647,0.596146


In [0]:
X = expander.iloc[:,1:].values
Y = y.values
y.shape, X.shape

((296450,), (296450, 6))

Splitting data. Train and test.

In [0]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.33,random_state=2, shuffle=True)

Custom function to calculate RMSE, which will be used for LSTM Model.

In [0]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 


Using TensorFlow backend.


A function to calculate RMSE, Accuracy, MAE after the predictions are obtained.

In [0]:
def metric_display(preds):
  errors = abs(preds - y_test)
  final_rmse = np.sqrt(np.mean(errors*errors))
  mape = 100 * (errors / y_test)
  accuracy = 100 - np.mean(mape)
  return "Mean Absolute Error: ", round(np.mean(errors), 2), "RMSE: ", final_rmse, "Accuracy: ", round(accuracy, 2)


Exploring Machine Learning/Deep Learning Models.

1. Random Forest

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators= 100, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [0]:
preds_rf = rf.predict(X_test)
scores_rf = metric_display(preds_rf)
print(scores_rf)

('Mean Absolute Error: ', 1.08, 'RMSE: ', 1.342022555464408, 'Accuracy: ', 99.42)


2. Gradient Boost

In [0]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators= 100, random_state=42)
gb.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
preds_gb = gb.predict(X_test)
scores_gb = metric_display(preds_gb)
print(scores_gb)

('Mean Absolute Error: ', 1.79, 'RMSE: ', 2.3616757284151517, 'Accuracy: ', 99.01)


3. AdaBoost

In [0]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor(n_estimators= 200, random_state=42, learning_rate=0.1)
ada.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.1, loss='linear',
                  n_estimators=200, random_state=42)

In [0]:
preds_ada = ada.predict(X_test)
scores_ada = metric_display(preds_ada)
print(scores_ada)


('Mean Absolute Error: ', 6.68, 'RMSE: ', 7.968433090920889, 'Accuracy: ', 96.09)


4. SVM

In [0]:
from sklearn.svm import SVR
svm=SVR()
svm.fit(X_train,y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
preds_svm = svm.predict(X_test)
scores_svm = metric_display(preds_svm)
print(scores_svm)


('Mean Absolute Error: ', 1.04, 'RMSE: ', 1.2601093235915026, 'Accuracy: ', 99.43)


Deep Learning Model LSTM

In [0]:
X_train_lst=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test_lst=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [0]:
from pandas import read_csv
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Input, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

model = Sequential()
inp = Input(shape=(6,1))
x = LSTM(25, activation='relu', dropout=0.02)(inp)
x = Dense(1, kernel_initializer='normal')(x)
model = Model(inputs=inp, outputs=x)
model.compile(optimizer='adam', loss = root_mean_squared_error, metrics = ['mean_absolute_error'])
model.fit(x=X_train_lst, y=y_train, batch_size=150, epochs=50, verbose=1)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/50
198621/198621 [==============================] - 17s 86us/step - loss: 31.1673 - mean_absolute_error: 25.8004
Epoch 2/50
198621/198621 [==============================] - 16s 83us/step - loss: 8.6681 - mean_absolute_error: 5.5626
Epoch 3/50
198621/198621 [==============================] - 17s 88us/step - loss: 6.3963 - mean_absolute_error: 4.4562
Epoch 4/50
198621/198621 [==============================] - 17s 84us/step - loss: 5.2528 - mean_absolute_error: 3.7538
Epoch 5/50
198621/198621 [==============================] - 16s 82us/step - loss: 4.7517 - mean_absolute_error: 3.3540
Epoch 6/50
198621/198621 [==============================] - 16s 83us/step - loss: 4.3982 - mean_absolute_error: 3.0911
Epoch 7/50
198621/198621 [==============================] - 16s 82us/step - loss: 4.0700 - mean_absolute_error: 2.8401
Epoch 8/50
198621/198621 [=======================

In [0]:
preds = model.predict(X_test_lst)

In [0]:
preds = preds.tolist()
preds = [item for sublist in preds for item in sublist]
preds = np.array(preds)

scores_lstm = metric_display(preds)
print(scores_lstm)

('Mean Absolute Error: ', 1.66, 'RMSE: ', 1.9576741790350358, 'Accuracy: ', 98.93)


In [0]:
dictionary = {"MAE":[],"RMSE":[],"Accuracy":[]}
scores
for i in range(len(scores_ada)):
  if i==1:
    dictionary["MAE"].append(scores_ada[i])
    dictionary["MAE"].append(scores_gb[i])
    dictionary["MAE"].append(scores_lstm[i])
    dictionary["MAE"].append(scores_rf[i])
    dictionary["MAE"].append(scores_svm[i])
  elif i==3:
    dictionary["RMSE"].append(scores_ada[i])
    dictionary["RMSE"].append(scores_gb[i])
    dictionary["RMSE"].append(scores_lstm[i])
    dictionary["RMSE"].append(scores_rf[i])
    dictionary["RMSE"].append(scores_svm[i])
  elif i==5:
    dictionary["Accuracy"].append(scores_ada[i])
    dictionary["Accuracy"].append(scores_gb[i])
    dictionary["Accuracy"].append(scores_lstm[i])
    dictionary["Accuracy"].append(scores_rf[i])
    dictionary["Accuracy"].append(scores_svm[i])

dictionary

{'Accuracy': [96.09, 99.01, 98.93, 99.42, 99.43],
 'MAE': [6.68, 1.79, 1.66, 1.08, 1.04],
 'RMSE': [7.968433090920889,
  2.3616757284151517,
  1.9576741790350358,
  1.342022555464408,
  1.2601093235915026]}

In [0]:
models = ["AdaBoost","GradientBoost","LSTM","RandomForest","SVM"]
result_df = pd.DataFrame.from_dict(dictionary,orient='index').transpose()
result_df["models"] = models
result_df = result_df[['models', 'MAE', 'RMSE', 'Accuracy',]]
result_df = result_df.sort_values('Accuracy')


Final result, SVM seems to overperform every other Machine learning model as well as LSTM. Maybe if the dataset had more attributes (or/and) the dataset was larger, LSTM would have obtained better results.

In [0]:
result_df

,models,MAE,RMSE,Accuracy
0,AdaBoost,6.68,7.968433,96.09
2,LSTM,1.66,1.957674,98.93
1,GradientBoost,1.79,2.361676,99.01
3,RandomForest,1.08,1.342023,99.42
4,SVM,1.04,1.260109,99.43
